# Calling Keras Layers on TensorFlow tensors

* https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

Input은 TensorFlow를 사용하고, Keras에서는 Layer구성및 categorical crossentropy를 구성함. 

### Different behaviors during training and testing 

Keras Layers등중 몇몇 (Dropout, BatchNormalization)등은 Training과 Testing Time때 다르게 작동을 합니다. <br>
<span style="color:red">**keras.backend.learning_phase()**</span> 를 feed_dict에 함께 써줘야 합니다. 

* backend.learning_phase(): True -> Training
* backend.learning_phase(): False -> Testing


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

from keras.layers import  Dense, Dropout
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy
from keras import 

import numpy as np

Using TensorFlow backend.


### Create a Session

In [2]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.4, allow_growth=True)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))
backend.set_session(sess)

### Data

In [3]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Model

In [4]:
images = tf.placeholder(tf.float32, shape=(None, 784))
labels = tf.placeholder(tf.float32, shape=(None, 10))

# Keras Layer는 TensorFlow tensor를 받을수 있습니다. 
layer = Dense(128, activation='relu')(images)
layer = Dropout(0.5)(layer)
layer = Dense(128, activation='relu')(layer)
layer = Dropout(0.5)(layer)
predicts = Dense(10, activation='softmax')(layer)

# Training 
loss = tf.reduce_mean(categorical_crossentropy(labels, predicts))
train_step = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

### Train

In [14]:
with sess.as_default():
    tf.global_variables_initializer().run()
    for i in range(20000):
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={images: batch[0], labels:batch[1], 
                                  backend.learning_phase(): True})

### Evaluate

In [15]:
accrucy = categorical_accuracy(labels, predicts)
with sess.as_default():
    print(accrucy.eval(feed_dict={images: mnist.test.images, labels: mnist.test.labels,
                                  backend.learning_phase(): False}))

0.9557


In [16]:
yhat = np.argmax(predicts.eval(feed_dict={images: mnist.test.images, 
                                          backend.learning_phase(): False}), axis=1)
correct = np.argmax(mnist.test.labels, axis=1)
len(correct[yhat == correct])/len(correct)

0.9557